In [1]:
import os
import os.path as ops
import argparse
import time
import math

import tensorflow as tf
import glob
import glog as log
import numpy as np
import matplotlib.pyplot as plt
import cv2

from lanenet_model import lanenet_merge_model
from lanenet_model import lanenet_cluster
from lanenet_model import lanenet_postprocess
from config import global_config

In [2]:
CFG = global_config.cfg
VGG_MEAN = [103.939, 116.779, 123.68]

In [3]:
CFG

{'TEST': {'BATCH_SIZE': 8,
  'GPU_MEMORY_FRACTION': 0.8,
  'TF_ALLOW_GROWTH': True},
 'TRAIN': {'BATCH_SIZE': 8,
  'CLASSES_NUMS': 2,
  'DISPLAY_STEP': 10,
  'EPOCHS': 200010,
  'GPU_MEMORY_FRACTION': 0.9,
  'IMG_HEIGHT': 256,
  'IMG_WIDTH': 512,
  'LEARNING_RATE': 0.0005,
  'LR_DECAY_RATE': 0.1,
  'LR_DECAY_STEPS': 80000,
  'MOMENTUM': 0.9,
  'TEST_DISPLAY_STEP': 100,
  'TF_ALLOW_GROWTH': True,
  'VAL_BATCH_SIZE': 8}}

In [4]:
def minmax_scale(input_arr):
    """

    :param input_arr:
    :return:
    """
    min_val = np.min(input_arr)
    max_val = np.max(input_arr)

    output_arr = (input_arr - min_val) * 255.0 / (max_val - min_val)
    return output_arr

In [5]:
def test_lanenet_batch(image_dir, weights_path, batch_size, use_gpu, save_dir=None):
    """h 

    :param image_dir:
    :param weights_path:
    :param batch_size:
    :param use_gpu:
    :param save_dir:
    :return:
    """
    assert ops.exists(image_dir), '{:s} not exist'.format(image_dir)

    log.info('Start getting the image file path...')
    image_path_list = glob.glob('{:s}/**/*.jpg'.format(image_dir), recursive=True) + \
                      glob.glob('{:s}/**/*.png'.format(image_dir), recursive=True) + \
                      glob.glob('{:s}/**/*.jpeg'.format(image_dir), recursive=True)

    input_tensor = tf.placeholder(dtype=tf.float32, shape=[None, 256, 512, 3], name='input_tensor')
    phase_tensor = tf.constant('test', tf.string)

    net = lanenet_merge_model.LaneNet(phase=phase_tensor, net_flag='vgg')
    binary_seg_ret, instance_seg_ret = net.inference(input_tensor=input_tensor, name='lanenet_model')

    cluster = lanenet_cluster.LaneNetCluster()
    postprocessor = lanenet_postprocess.LaneNetPoseProcessor()

    saver = tf.train.Saver()

    # Set sess configuration
    if use_gpu:
        sess_config = tf.ConfigProto(device_count={'GPU': 1})
    else:
        sess_config = tf.ConfigProto(device_count={'GPU': 0})
        
    sess_config.gpu_options.per_process_gpu_memory_fraction = CFG.TEST.GPU_MEMORY_FRACTION
    sess_config.gpu_options.allow_growth = CFG.TRAIN.TF_ALLOW_GROWTH
    sess_config.gpu_options.allocator_type = 'BFC'

    sess = tf.Session(config=sess_config)

    with sess.as_default():

        saver.restore(sess=sess, save_path=weights_path)

        epoch_nums = int(math.ceil(len(image_path_list) / batch_size))

        for epoch in range(epoch_nums):
            log.info('[Epoch:{:d}] Start image reading and preprocessing...'.format(epoch))
            
            t_start = time.time()
            image_path_epoch = image_path_list[epoch * batch_size:(epoch + 1) * batch_size]
            image_list_epoch = [cv2.imread(tmp, cv2.IMREAD_COLOR) for tmp in image_path_epoch]
            image_vis_list = image_list_epoch
            image_list_epoch = [cv2.resize(tmp, (512, 256), interpolation=cv2.INTER_LINEAR)
                                for tmp in image_list_epoch]
            image_list_epoch = [tmp - VGG_MEAN for tmp in image_list_epoch]
            t_cost = time.time() - t_start
            log.info('[Epoch:{:d}] Pre-processing {:d} images, total time consuming: {:.5f}s, \
            Average time per sheet: {:.5f}'.format(
                epoch, len(image_path_epoch), t_cost, t_cost / len(image_path_epoch)))

            t_start = time.time()
            binary_seg_images, instance_seg_images = sess.run(
                [binary_seg_ret, instance_seg_ret], feed_dict={input_tensor: image_list_epoch})
#             print(binary_seg_images.shape, instance_seg_ret.shape)
            
            
            t_cost = time.time() - t_start
            log.info('[Epoch:{:d}] Predicting {:d} image lane lines, total time: {:.5f}s, \
            average per time: {:.5f}s'.format(
                epoch, len(image_path_epoch), t_cost, t_cost / len(image_path_epoch)))

            cluster_time = []
            for index, binary_seg_image in enumerate(binary_seg_images):
                t_start = time.time()
                binary_seg_image = postprocessor.postprocess(binary_seg_image)
                mask_image = cluster.get_lane_mask(binary_seg_ret=binary_seg_image,
                                                   instance_seg_ret=instance_seg_images[index])
                cluster_time.append(time.time() - t_start)
                mask_image = cv2.resize(mask_image, (image_vis_list[index].shape[1],
                                                     image_vis_list[index].shape[0]),
                                        interpolation=cv2.INTER_NEAREST)
                
                image_name = ops.split(image_path_epoch[index])[1]
                image_save_path = ops.join(save_dir, image_name)
                image_save_path = ops.splitext(image_save_path)[0]
                
                _instance_seg_images = np.copy(instance_seg_images)
#                 print("mask", mask_image.shape)
                
                for i in range(4):
                    _instance_seg_images[index][:, :, i] = minmax_scale(instance_seg_images[index][:, :, i])
                _embedding_image = np.array(_instance_seg_images[index], np.uint8)
#                 print("embedding", _embedding_image.shape)

#                 print(np.max(instance_seg_images))
#                 print(np.min(instance_seg_images))
#                 m_act = np.max(instance_seg_images)
#                 cv2.imwrite(image_save_path + "embedding_{}.png".format(index), _embedding_image[:, :, (2, 1, 0)])
#                 cv2.imwrite(image_save_path + "binary_{}.png".format(index), binary_seg_image[:, :] * 255)
                
                if save_dir is None:
                    plt.imshow(mask_image[:, :, (2, 1, 0)])
                    plt.figure('src_image')
                    plt.imshow(image_vis_list[index][:, :, (2, 1, 0)])
                    plt.pause(3.0)
                    plt.show()
                    plt.ioff()

                if save_dir is not None:
                    mask_image = cv2.addWeighted(image_vis_list[index], 1.0, mask_image, 1.0, 0)
                    image_name = ops.split(image_path_epoch[index])[1]
                    image_save_path = ops.join(save_dir + "/image", image_name)
                    mask_save_path = ops.join(save_dir + "/mask", image_name)
                    embedding_save_path = ops.join(save_dir + "/embedding", image_name)
                    cv2.imwrite(mask_save_path, binary_seg_image * 255)
                    cv2.imwrite(image_save_path, mask_image)
                    cv2.imwrite(embedding_save_path, _embedding_image[:, :, (2, 1, 0)])

            log.info('[Epoch:{:d}] Perform {:d} image lane line clustering, total time: {:.5f}s, \
            average time per sheet: {:.5f}'.format(
                epoch, len(image_path_epoch), np.sum(cluster_time), np.mean(cluster_time)))

    sess.close()
    tf.reset_default_graph()

return

In [6]:
import cv2
import numpy as np
from sklearn.linear_model import LinearRegression

H_SAMPLES = np.array([160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360,
             370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570,
             580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710])

lr = LinearRegression()


def get_angle(xs, y_samples):
    xs, ys = xs[xs >= 0], y_samples[xs >= 0]
    if len(xs) > 1:
        lr.fit(ys[:, None], xs)
        k = lr.coef_[0]
        theta = np.arctan(k)
    else:
        theta = 0
    return theta


def get_line_representation(mask, line_value):
    line = []
    for h in H_SAMPLES:
        indices = np.where(mask[h] == line_value)
        if len(indices[0]) > 0:
            line.append(indices[0][int(len(indices[0]) / 2)])
        else:
            line.append(-2)
    return line


def get_13_masks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    values = np.unique(gray)
    print("Unique values: ", values)

    masks = np.zeros((13, np.shape(gray)[0], np.shape(gray)[1]))
    masks[0] = np.where(gray == 0, 1, 0)
    
    left_lines = []
    right_lines = []

    for value in values:
        if value == 0:
            continue

        line = get_line_representation(gray, value)

        angle = get_angle(np.array(line), H_SAMPLES)

        if angle < 0:
            left_lines.append([angle, value])
        else:
            right_lines.append([angle, value])

    left_lines.sort(key=lambda l: -l[0])
    right_lines.sort(key=lambda l: l[0])

    idx = 4
    for line in left_lines:
        value = line[1]
        masks[idx] = np.where(gray == value, 1, 0)
        idx -= 1

    idx = 6
    for line in right_lines:
        value = line[1]
        masks[idx] = np.where(gray == value, 1, 0)
        idx += 1

    return masks

In [7]:
from lanenet_model import lanenet_merge_model
from lanenet_model import lanenet_cluster
from lanenet_model import lanenet_postprocess
def test_lanenet(image_path, weights_path, use_gpu):
    """

    :param image_path:
    :param weights_path:
    :param use_gpu:
    :return:
    """
    assert ops.exists(image_path), '{:s} not exist'.format(image_path)

    log.info('Start reading image data and pre-processing')
    t_start = time.time()
    # image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    # copy image
    image_vis = image
    image = cv2.resize(image, (512, 256), interpolation=cv2.INTER_LINEAR)
#     plt.figure('image')
#     plt.imshow(image[:, :, (2, 1, 0)])
#     plt.show()
#     # normalisation. How to compute it??? Average on the sum on each channel for each image in the dataset???
#     image = image - VGG_MEAN
#     plt.figure('image-vgg')
#     plt.imshow(image[:, :, (2, 1, 0)])
#     plt.show()
    log.info('Image is read, time consuming: {:.5f}s'.format(time.time() - t_start))

    # take one image
    input_tensor = tf.placeholder(dtype=tf.float32, shape=[1, 256, 512, 3], name='input_tensor')
    
    # update the network parameters? test == No, train == Yes
    phase_tensor = tf.constant('test', tf.string)
    
    # does it containes the LSTM part???
    net = lanenet_merge_model.LaneNet(phase=phase_tensor, net_flag='vgg')
    
    # binary_seg_ret - lane or not-lane image
    # instance_seg_ret - pixel embedding ????
    binary_seg_ret, instance_seg_ret = net.inference(input_tensor=input_tensor, name='lanenet_model')
#     print(binary_seg_ret.shape)
#     print(instance_seg_ret.shape)
    
    #plt.figure('binary_seg_ret')
    
    #plt.imshow(binary_seg_ret[0])
    #     plt.figure('instance_seg_ret0')
#     plt.imshow(instance_seg_ret[0][:,:,0])
#     plt.figure('instance_seg_ret1')
#     plt.imshow(instance_seg_ret[0][:,:,1])
#     plt.figure('instance_seg_ret2')
#     plt.imshow(instance_seg_ret[0][:,:,2])
#     plt.figure('instance_seg_ret3')
#     plt.imshow(instance_seg_ret[0][:,:,3])
    # binary_seg_ret = tf.Print(binary_seg_ret, [binary_seg_ret[0]], message="binary_seg [0]")

    cluster = lanenet_cluster.LaneNetCluster()
    postprocessor = lanenet_postprocess.LaneNetPoseProcessor()

    saver = tf.train.Saver()

    # Set sess configuration
    if use_gpu:
        sess_config = tf.ConfigProto(device_count={'GPU': 0})
    else:
        sess_config = tf.ConfigProto(device_count={'CPU': 0})
    
    # prevent using the entire gpu memory for training/inference
    sess_config.gpu_options.per_process_gpu_memory_fraction = CFG.TEST.GPU_MEMORY_FRACTION
    
    # don't allocate the entire available memory from the start. 
    sess_config.gpu_options.allow_growth = CFG.TRAIN.TF_ALLOW_GROWTH
    sess_config.gpu_options.allocator_type = 'BFC'

    sess = tf.Session(config=sess_config)

    with sess.as_default():

        saver.restore(sess=sess, save_path=weights_path)

        t_start = time.time()
       
        binary_seg_image, instance_seg_image = sess.run([binary_seg_ret, instance_seg_ret],
                                                        feed_dict={input_tensor: [image]})
        
        _instance_seg_image = np.copy(instance_seg_image)
        for i in range(4):
            _instance_seg_image[0][:, :, i] = minmax_scale(instance_seg_image[0][:, :, i])
        _embedding_image = np.array(_instance_seg_image[0], np.uint8)

#         plt.figure('src_image')
#         plt.imshow(image_vis[:, :, (2, 1, 0)])
#         plt.figure('instance_image')
#         plt.imshow(_embedding_image[:, :, (2, 1, 0)])
#         plt.figure('binary_image')
#         plt.imshow(binary_seg_image[0] * 255, cmap='gray')
#         plt.show()
        
        t_cost = time.time() - t_start
        log.info('Single image lane line prediction time consuming: {:.5f}s'.format(t_cost))

        # remove connected components that are smaller 15 pixels
        binary_seg_image[0] = postprocessor.postprocess(binary_seg_image[0], minarea_threshold=15)
        mask_image = cluster.get_lane_mask(binary_seg_ret=binary_seg_image[0],
                                           instance_seg_ret=instance_seg_image[0])

        for i in range(4):
            instance_seg_image[0][:, :, i] = minmax_scale(instance_seg_image[0][:, :, i])
        embedding_image = np.array(instance_seg_image[0], np.uint8)
#         print("hereeee")
#         print(embedding_image.shape)
        plt.figure('mask_image')
        plt.imshow(mask_image[:, :, (2, 1, 0)])
        plt.figure('src_image')
        plt.imshow(image_vis[:, :, (2, 1, 0)])
        plt.figure('instance_image')
        plt.imshow(embedding_image[:, :, (2, 1, 0)])
        plt.figure('binary_image')
        plt.imshow(binary_seg_image[0] * 255, cmap='gray')
        plt.show()
        
        print(mask_image.shape)
        mask_image = cv2.resize(mask_image, dsize=(1280, 720),
                   interpolation=cv2.INTER_NEAREST)
        print(mask_image.shape)
        
        masks = get_13_masks(mask_image)
        
        for i in range(13):
            plt.figure('mask' + str(i))
            plt.imshow(masks[i])
        plt.show()

    sess.close()
    tf.reset_default_graph() 

return

In [8]:
image_path = '/workspace/storage/datasets/AVMSnapshots/AVM/val_images'
weights_path = '/workspace/storage/projects/lanenet-lane-detection/AVM_resized_train/tusimple_lanenet_vgg_2019-03-26-09-12-58.ckpt-200000'
is_batch = True
batch_size = 32
save_dir = '/workspace/storage/datasets/AVMSnapshots/AVM/output_val_images_preprocess'
use_gpu = 1

if save_dir is not None and not ops.exists(save_dir):
    log.error('{:s} not exist and has been made'.format(save_dir))
    os.makedirs(save_dir)
    os.makedirs(save_dir + "/image")
    os.makedirs(save_dir + "/mask")
    os.makedirs(save_dir + "/embedding")

if is_batch == False:
    # test hnet model on single image
    test_lanenet(image_path, weights_path, use_gpu)
else:
    # test hnet model on a batch of image
    test_lanenet_batch(image_dir=image_path, weights_path=weights_path,
                       save_dir=save_dir, use_gpu=use_gpu, batch_size=batch_size)

E0328 12:09:38.907010 19950 <ipython-input-8-1986fdf3baa6>:9] /workspace/storage/datasets/AVMSnapshots/AVM/output_val_images_preprocess not exist and has been made
I0328 12:09:38.908232 19950 <ipython-input-5-76cb2322defa>:16] Start getting the image file path...



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


W0328 12:09:39.618025 19950 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use keras.layers.batch_normalization instead.


W0328 12:09:39.622740 19950 deprecation.py:323] From /workspace/storage/projects/lanenet-lane-detection/encoder_decoder_model/cnn_basenet.py:316: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.batch_normalization instead.


...

W0328 12:09:39.955017 19950 deprecation.py:323] From /workspace/storage/projects/lanenet-lane-detection/encoder_decoder_model/cnn_basenet.py:370: conv2d_transpose (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


W0328 12:09:40.683717 19950 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /workspace/storage/projects/lanenet-lane-detection/AVM_resized_train/tusimple_lanenet_vgg_2019-03-26-09-12-58.ckpt-200000


I0328 12:09:40.690546 19950 saver.py:1270] Restoring parameters from /workspace/storage/projects/lanenet-lane-detection/AVM_resized_train/tusimple_lanenet_vgg_2019-03-26-09-12-58.ckpt-200000
I0328 12:09:40.811508 19950 <ipython-input-5-76cb2322defa>:51] [Epoch:0] Start image reading and preprocessing...
I0328 12:09:41.513303 19950 <ipython-input-5-76cb2322defa>:63] [Epoch:0] Pre-processing 32 images, total time consuming: 0.70125s,                     Average time per sheet: 0.02191
I0328 12:09:44.497611 19950 <ipython-input-5-76cb2322defa>:74] [Epoch:0] Predicting 32 image lane lines, total time: 2.98370s,                     average per time: 0.09324s
I0328 12:09:44.637300 19950 lanenet_cluster.py:81] 聚类簇个数为: 4
W0328 12:09:44.669817 19950 <ipython-input-4-b990e2c01189>:10] /usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.

I0328 12:09:44.779288 19950 lanenet_c